In [28]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import joblib  # Import joblib for model saving

In [56]:
# Load the training dataset
train_data = pd.read_csv('train_data.csv', delimiter=';')

In [57]:
# Drop unnecessary columns
train_data = train_data.drop(['PlayerURL', 'PlayerName'], axis=1)
train_data.head()
#train_data.tail()

,PlayerID,Race,Move 1,Move 2,Move 3,Move 4,Move 5,Move 6,Move 7,Move 8,...,Move 2554,Move 2555,Move 2556,Move 2557,Move 2558,Move 2559,Move 2560,Move 2561,Move 2562,Move 2563
0,1021189,Terran,s,hotkey11,hotkey21,hotkey31,hotkey41,hotkey51,hotkey61,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1021189,Terran,s,s,hotkey11,hotkey21,hotkey31,hotkey41,hotkey51,hotkey61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1021189,Terran,s,hotkey11,hotkey21,hotkey31,hotkey41,hotkey51,hotkey61,hotkey71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1021189,Terran,s,hotkey11,hotkey21,hotkey31,hotkey41,hotkey51,hotkey61,t5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1021189,Terran,s,hotkey11,hotkey21,hotkey31,hotkey41,hotkey51,hotkey71,hotkey61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# new table that only contains the first two columns (PlayerId and Race) of train_data

# Keep only the first two columns but all rows
train_data_new = train_data.iloc[:, :2]

#test
train_data_new.head()

,PlayerID,Race
0,1021189,Terran
1,1021189,Terran
2,1021189,Terran
3,1021189,Terran
4,1021189,Terran


In [74]:
def count_moves(row, counts, index):
    for i in range(1, 2564):
        move = row["Move "+ str(i)]

        # count the number of s's
        if move == 's':
            counts[10][index] += 1
        # count the number of Base's
        elif move == 'Base':
            counts[11][index] += 1
        # count the number of SingleMineral's
        elif move == 'SingleMineral':
            counts[12][index] += 1
        # count the hotkeys
        elif isinstance(move, str):
            for j in range(10):
                if move.startswith(f"hotkey{j}"):
                    counts[j][index] += 1


def count_move_per_time(row, counts, row_index, time_interval, ti_index):
    base_index = ti_index*13

    for i in range(1, 2564):
        move = row["Move "+ str(i)]
        
        if move == f't{time_interval}':
            return;
        
        else:
            # Count actions for the given time interval
            if move == 's':
                counts[base_index+10][row_index] += 1
            elif move == 'Base':
                counts[base_index+11][row_index] += 1
            elif move == 'SingleMineral':
                counts[base_index+12][row_index] += 1

            # Count hotkeys for the given time interval
            elif isinstance(move, str):
                for j in range(10):
                    if move.startswith(f"hotkey{j}_t{time_interval}"):
                        counts[base_index+j][row_index] += 1


# add the count of Moves per row

# new lists of counts
counts = [[0] * 3052 for _ in range(65)]

# Specify the target time intervals
time_intervals = [20, 40, 45, 145]

# go through the rows
for row_index, row in train_data.iterrows():
    count_moves(row, counts, row_index)

    for ti_index, time_interval in enumerate(time_intervals):
        count_move_per_time(row, counts, row_index, time_interval, ti_index+1)


for i in range(10):
    train_data_new[f'hk{i}Counts'] = counts[i]
    
train_data_new['sCounts'] = counts[10]
train_data_new['baseCounts'] = counts[11]
train_data_new['singleMineralCounts'] = counts[12]

for ti_index, time_interval in enumerate(time_intervals):
    base_index = (ti_index+1)*13
    for j in range(10):
        train_data_new[f'hk{j}_t{time_interval}_Counts'] = counts[base_index + j]

    train_data_new[f's_t{time_interval}_Counts'] = counts[base_index + 10]
    train_data_new[f'base_t{time_interval}_Counts'] = counts[base_index + 11]
    train_data_new[f'singleMineral_t{time_interval}_Counts'] = counts[base_index + 12]
    

train_data_new.head()

0
20
1
40
2
45
3
145


C:\Users\thekl\AppData\Local\Temp\ipykernel_12676\2837306854.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data_new['hk' + str(j) + '_t' + str(time_interval) + '_Counts'] = counts[base_index + j]
C:\Users\thekl\AppData\Local\Temp\ipykernel_12676\2837306854.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data_new['hk' + str(j) + '_t' + str(time_interval) + '_Counts'] = counts[base_index + j]
C:\Users\thekl\AppData\Local\Temp\ipykernel_12676\2837306854.py:74: PerformanceWarning: DataFrame is highly fragment

,PlayerID,Race,hk0Counts,hk1Counts,hk2Counts,hk3Counts,hk4Counts,hk5Counts,hk6Counts,hk7Counts,...,hk3_t145_Counts,hk4_t145_Counts,hk5_t145_Counts,hk6_t145_Counts,hk7_t145_Counts,hk8_t145_Counts,hk9_t145_Counts,s_t145_Counts,base_t145_Counts,singleMineral_t145_Counts
0,1021189,Terran,0,405,275,83,77,20,1,0,...,0,0,0,0,0,0,0,188,0,1
1,1021189,Terran,0,382,159,141,44,5,1,0,...,0,0,0,0,0,0,0,135,0,0
2,1021189,Terran,0,334,222,48,79,16,1,1,...,0,0,0,0,0,0,0,138,0,2
3,1021189,Terran,0,427,309,86,114,55,3,0,...,0,0,0,0,0,0,0,181,0,1
4,1021189,Terran,0,396,198,58,107,28,1,1,...,0,0,0,0,0,0,0,151,0,1


In [75]:
df = train_data_new[['PlayerID','singleMineral_t20_Counts', 'singleMineral_t40_Counts', 'singleMineral_t145_Counts']]
print(df)

      PlayerID  singleMineral_t20_Counts  singleMineral_t40_Counts  \
0      1021189                         1                         1   
1      1021189                         0                         0   
2      1021189                         0                         1   
3      1021189                         0                         0   
4      1021189                         0                         0   
...        ...                       ...                       ...   
3047       415                         0                         0   
3048       415                         0                         0   
3049       415                         0                         0   
3050       415                         0                         0   
3051       415                         0                         0   

      singleMineral_t145_Counts  
0                             1  
1                             0  
2                             2  
3                      

In [43]:
print(train_data_new['s_t100_Counts'])

0       1245
1        458
2        879
3        956
4        927
        ... 
3047     785
3048    1096
3049    1114
3050     987
3051    1065
Name: s_t100_Counts, Length: 3052, dtype: int64


In [27]:
# add the count of Moves per row

# new lists of counts
counts = [[0] * 3052 for _ in range(13)]

# go through the rows
for index, row in train_data.iterrows():

    # go through the moves 
    for i in range(1, 2564):
        
        # count the number of s's
        if row["Move "+ str(i)] == 's':
            counts[10][index] += 1
        
        # count the number of Base's
        elif row["Move "+ str(i)] == 'Base':
            counts[11][index] += 1
        
        # count the number of SingleMineral's
        elif row["Move "+ str(i)] == 'SingleMineral':
            counts[12][index] += 1
            
            
        # count the hotkeys
        elif isinstance(row["Move "+ str(i)], str):
            
            # count the number of hotkey0_'s
            if row["Move "+ str(i)].startswith("hotkey0"):
                counts[0][index] += 1
            
            # count the number of hotkey1_'s
            elif row["Move "+ str(i)].startswith("hotkey1"):
                counts[1][index] += 1
                
            # count the number of hotkey2_'s
            elif row["Move "+ str(i)].startswith("hotkey2"):
                counts[2][index] += 1
                
            # count the number of hotkey3_'s
            elif row["Move "+ str(i)].startswith("hotkey3"):
                counts[3][index] += 1
                
            # count the number of hotkey4_'s
            elif row["Move "+ str(i)].startswith("hotkey4"):
                counts[4][index] += 1
                
            # count the number of hotkey5_'s
            elif row["Move "+ str(i)].startswith("hotkey5"):
                counts[5][index] += 1
                
            # count the number of hotkey6_'s
            elif row["Move "+ str(i)].startswith("hotkey6"):
                counts[6][index] += 1
                
            # count the number of hotkey7_'s
            elif row["Move "+ str(i)].startswith("hotkey7"):
                counts[7][index] += 1
                
            # count the number of hotkey8_'s
            elif row["Move "+ str(i)].startswith("hotkey8"):
                counts[8][index] += 1
                
            # count the number of hotkey9_'s
            elif row["Move "+ str(i)].startswith("hotkey9"):
                counts[9][index] += 1
            
            

for i in range(10):
    train_data_new['hk' + str(i) + 'Counts'] = counts[i]

train_data_new['sCounts'] = counts[10]
train_data_new['baseCounts'] = counts[11]
train_data_new['singleMineralCounts'] = counts[12]

train_data_new.head()

,PlayerID,Race,hk0Counts,hk1Counts,hk2Counts,hk3Counts,hk4Counts,hk5Counts,hk6Counts,hk7Counts,hk8Counts,hk9Counts,sCounts,baseCounts,singleMineralCounts
0,1021189,Terran,0,405,275,83,77,20,1,0,0,0,1245,0,2
1,1021189,Terran,0,382,159,141,44,5,1,0,0,0,458,0,0
2,1021189,Terran,0,334,222,48,79,16,1,1,0,0,879,0,8
3,1021189,Terran,0,427,309,86,114,55,3,0,0,0,956,0,9
4,1021189,Terran,0,396,198,58,107,28,1,1,0,0,927,0,3


In [65]:
# Create an empty list to store counts for each row
row_action_counts = []

# Iterate through each row of the dataframe
for _, row in train_data.iterrows():
    # Initialize a counter for each time window
    action_count_before_time = 0

    # Initialize a dictionary to store counts for each time window
    counts_before_100 = {}

    count_100 = 1

    # Iterate through each 'Move_XX' column for the current row
    for col in train_data.columns[3:]: 
        # Check if the value is different from -1
        if row[col] != -1:
            action_count_before_time += 1

            # Check if the value is 100
            if row[col] == 100:
                timestamp = count_100 * 5 
                counts_before_100[f't{timestamp}'] = action_count_before_time
                action_count_before_time = -1 
                count_100 += 1

    #If no action is found, set count to 0
    if not action_count_before_time:
        action_count_before_time = 0

    # Append the counts for the current row to the list
    row_action_counts.append(counts_before_100)

In [66]:
# Create a csv from the results
result_df = pd.DataFrame(row_action_counts)

# Add the 'PlayerID' and 'Race' column
result_df.insert(0, 'PlayerID', train_data['PlayerID'])
result_df.insert(1, 'Race', train_data['Race'])

# Save the DataFrame to a CSV file
result_df.to_csv('move_count.csv', index=False)

In [67]:
train_data = pd.read_csv('move_count.csv')

In [68]:
train_data.head()

,PlayerID,Race,t5,t10,t15,t20,t25,t30,t35,t40,...,t2125,t2130,t2135,t2140,t2145,t2150,t2155,t2160,t2165,t2170
0,1021189,2,11.0,8.0,7.0,12.0,7.0,1.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1021189,2,8.0,9.0,12.0,1.0,11.0,9.0,21.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1021189,2,8.0,16.0,5.0,7.0,9.0,12.0,11.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1021189,2,7.0,15.0,20.0,10.0,15.0,9.0,7.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1021189,2,9.0,12.0,8.0,6.0,15.0,15.0,13.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# get all the player IDs that exist in one list

player = train_data['PlayerID']

player_ids = []

for pl in player:
    if pl not in player_ids:
        player_ids.append(pl)

#test
print(player_ids)

[1021189, 1058669, 1139573, 1143713, 1173786, 1430346, 2048063, 2101268, 2115876, 219517, 2222468, 2383348, 2385865, 239335, 2452136, 250458, 251061, 2526293, 2639840, 2685580, 284466, 2896854, 2898004, 2910724, 2978202, 3010872, 3074362, 3090974, 315813, 326029, 3311551, 3368730, 3401218, 3434150, 3437681, 349650, 3533538, 3538115, 3585588, 3611718, 3746323, 377576, 377903, 3880481, 3886231, 3971497, 3973341, 4064798, 4085747, 4149248, 4234852, 4341883, 4854541, 4860568, 4863808, 498165, 788178, 884897, 950504, 1106851, 1126978, 1178662, 2240191, 2332264, 2333068, 2333312, 2334176, 2334376, 2340350, 2341466, 2341467, 2341492, 2341545, 2341575, 2341700, 2341765, 2341920, 2342120, 2342147, 2342269, 2342294, 2342299, 2342343, 2342419, 2342491, 2342671, 2342766, 2342785, 2342789, 2342794, 2342923, 2342983, 2343012, 2343067, 2343183, 2343240, 2343263, 2343479, 2343484, 2343531, 2343559, 2343607, 2343733, 2343852, 2343882, 2343910, 2343979, 2344031, 2344079, 2344081, 2344151, 2344207, 23442

In [32]:
mg.head()

,PlayerID,Race,Move_1,Move_2,Move_3,Move_4,Move_5,Move_6,Move_7,Move_8,...,t2125,t2130,t2135,t2140,t2145,t2150,t2155,t2160,t2165,t2170
0,1021189,2,0,7,10,13,16,19,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1021189,2,0,7,10,13,16,19,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1021189,2,0,7,10,13,16,19,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1021189,2,0,7,10,13,16,19,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1021189,2,0,7,10,13,16,19,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,PlayerID,Race,Move_1,Move_2,Move_3,Move_4,Move_5,Move_6,Move_7,Move_8,...,Move_2554,Move_2555,Move_2556,Move_2557,Move_2558,Move_2559,Move_2560,Move_2561,Move_2562,Move_2563
0,1021189,2,0,7,10,13,16,19,22,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1021189,2,0,0,7,10,13,16,19,22,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1021189,2,0,7,10,13,16,19,22,25,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1021189,2,0,7,10,13,16,19,22,100,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1021189,2,0,7,10,13,16,19,25,22,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [70]:
# new table that only contains the first two columns (PlayerId and Race) of train_data

# Keep only the first two columns but all rows
small_train_data = train_data.iloc[:, :2]

#test
small_train_data.head()

,PlayerID,Race
0,1021189,2
1,1021189,2
2,1021189,2
3,1021189,2
4,1021189,2


In [13]:
# create a new table that has only one row for a Player playing a certain game

new_train_data = []

for index, row in train_data.iterrows():
    playerID = row['PlayerID']
    race = row['Race']
    
    if [playerID, race] not in new_train_data:
        new_train_data.append([playerID, race])
        
        



# #test 
print(new_train_data)

[[1021189, 'Terran'], [1058669, 'Terran'], [1139573, 'Protoss'], [1143713, 'Terran'], [1173786, 'Zerg'], [1430346, 'Protoss'], [2048063, 'Zerg'], [2101268, 'Zerg'], [2115876, 'Zerg'], [219517, 'Zerg'], [2222468, 'Zerg'], [2383348, 'Terran'], [2385865, 'Protoss'], [239335, 'Protoss'], [2452136, 'Protoss'], [250458, 'Zerg'], [251061, 'Zerg'], [2526293, 'Protoss'], [2639840, 'Zerg'], [2685580, 'Protoss'], [284466, 'Terran'], [2896854, 'Terran'], [2898004, 'Terran'], [2910724, 'Protoss'], [2978202, 'Terran'], [3010872, 'Protoss'], [3074362, 'Protoss'], [3090974, 'Terran'], [315813, 'Protoss'], [326029, 'Zerg'], [3311551, 'Zerg'], [3368730, 'Protoss'], [3401218, 'Protoss'], [3434150, 'Terran'], [3437681, 'Zerg'], [349650, 'Protoss'], [3533538, 'Protoss'], [3538115, 'Zerg'], [3585588, 'Protoss'], [3611718, 'Terran'], [3746323, 'Zerg'], [377576, 'Protoss'], [377903, 'Zerg'], [3880481, 'Protoss'], [3886231, 'Protoss'], [3971497, 'Protoss'], [3973341, 'Protoss'], [4064798, 'Terran'], [4085747, 

In [ ]:
features = train_data_new.drop("PlayerID")
target = "PlayerID"

x_train, x_val, y_train, y_val = train_test_split(features, target, test_size=0.20, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

# model_filename = 'player_id_prediction_model.pkl'
# joblib.dump(tree, model_filename)

# print(f"Model saved as {model_filename}")